In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF
from PyRegridding.Utils import MyConstants as Con


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri


# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( fu )
importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)

Rgas = Con.Rdry() # 287.0 # J K-1 kg-1
grav = Con.grav() # 9.8


In [ ]:

### Make a list of files
import yaml
#from box import Box


with open('configure_ana_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)


##########################
# CHOOSE DATE TO LOOK AT
############################
ymdPat = '2004-06-07-*'

######################################################
season = cfg['plot']['season']
valData  = cfg['validation']['data']
valYears = cfg['validation']['years']

x='ne240x2' #_smth'

xl= ['oldCTL', 'c153_ne120_x2' ,'ne240x2', 'newCTL' ] #'ne240x2_QxQsst']

exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 



print( exp, subd, Src, Hkey, Dst, useri )
print( ymdPat , season )
print( valData , valYears )

A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                 hsPat='cam.h1i' , ymdPat=ymdPat,verbose=True, open_dataset=True )
A['Src']=Src
A['Hkey']=Hkey





In [ ]:

am=A.X.hyam.values
ai=A.X.hyai.values
bm=A.X.hybm.values
bi=A.X.hybi.values
ps=A.X.PS.values
te=A.X.T.values


pmid,pint,delp=MkP.Pressure( am=am, bm=bm, ai=ai, bi=bi, ps=ps , p_00=100_000., Gridkey='tzc' )

Topo=xr.open_dataset( A.X.topography_file )
topo=Topo.PHIS/grav

z3e , z3o = MkP.GeopHeight( te, delp, pmid, topo=topo, Gridkey='tzc')

In [ ]:
og=A.X.OMEGA.values
uu=A.X.U.values
vv=A.X.V.values


In [ ]:
RgObs={}
RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=A.Src, dst='latlonOxO',RegridMethod='BILINEAR')


In [ ]:
uu_xO=RgF.Horz(xfld_Src=uu , RegridObj_In= RegridObj ) 
vv_xO=RgF.Horz(xfld_Src=vv , RegridObj_In= RegridObj ) 
og_xO=RgF.Horz(xfld_Src=og , RegridObj_In= RegridObj ) 
z3o_xO=RgF.Horz(xfld_Src=z3o , RegridObj_In= RegridObj ) 
z3e_xO=RgF.Horz(xfld_Src=z3e , RegridObj_In= RegridObj ) 
te_xO=RgF.Horz(xfld_Src=te , RegridObj_In= RegridObj ) 


cpair= Con.cpair()
dse_xO = cpair * te_xO + grav * z3o_xO


In [ ]:
# Only possible on lat-lon field


lat_xO,lon_xO =GrU.latlon(grid='latlonOxO' )
nt,nz,ny,nx=np.shape( uu_xO )

zeta_xO=np.zeros((nt,nz,ny,nx) )
for t in np.arange( nt ):
    for z in np.arange(nz):
        if ( (z%10)==0):
            print( f'time {t}, z {z}' ) 
        zeta_xO [t,z,:,:] = nuti.Sphere_Curl2( uu_xO[t,z,:,:] , vv_xO[t,z,:,:] , lat_xO, lon_xO, wrap=True )



og_xO_f=np.zeros((nt,nz, ny,nx) )
for t in np.arange( nt ):
    for z in np.arange( nz ):
        if ( (t%1)==0) and ( (z%10)==0):
            print( f'time {t} z {z}' ) 
        og_xO_f[t,z,:,:]=fu.filterk_np( og_xO[t,z,:,:] , kmin=0,kmax=20,dimlon=1 )




In [ ]:
print( ' .... stopping '  )

In [ ]:
print( ' .... stopping '  )

In [ ]:
print( ' .... stopping '  )

In [ ]:
Lon_xO = np.tile( lon_xO, (nt,nz,ny,1) )

In [ ]:
plt.contour( Lon_xO[0,0,:,:] )

In [ ]:
print(lat_xO[8*42] )

In [ ]:
plev=A.X.lev.values
zlev=-7.0*np.log(plev/1_000.)

t,z,y = 0,0,600

#co=plt.contourf( lon_xO, zlev, uu_xO[0,:,8*42,:] ,cmap='bwr')
c2=plt.contourf( Lon_xO[t,:,y,:], z3o_xO[t,:,y,:], og_xO[t,:,y,:] ,cmap='bwr',levels=51,alpha=1)
l2=plt.plot( Lon_xO[t,92,y,:],z3e_xO[t,93,y,:] )
plt.ylim( 0,20000) 
plt.colorbar(c2)

In [ ]:
plt.contourf( z3e_xO[t,93,:,:] )

In [ ]:
# Assuming your dataset is named 'ds' and the longitude coordinate is named 'lon'
#Topo['lon'] = xr.where(Topo['lon'] > 180, Topo['lon'] - 360, Topo['lon'])
# Roll the dataset along the longitude axis
#Topo = Topo.roll(lon=len(Topo['lon']) // 2, roll_coords=True)

In [ ]:
Topo2=xr.open_dataset("../AnimationFiles/Topo_ne240_OxO_remap.nc" )


topo2=Topo2.TopoHt.values
levels_topo=[1.,10.]  #,100.,500.,1000.,3000.,5000.]



In [ ]:
plt.contour(topo2,levels=[1,10,100])

In [ ]:
dog_xO = og_xO - og_xO_f
for z in np.arange( nz ):
    dog_xO[:,z,:,:] = (1./9.8)*(1_000./plev[z]) * dog_xO[:,z,:,:] 

In [ ]:
topo_xO = z3e_xO[0,93,:,:]

In [ ]:
target_plevs = [ 932.0, 856., 499., 227.0,  58., 2.8]
lev_sel = uti.find_nearest_plev_indices(plev=plev , target_levels=target_plevs )
lat_sel= uti.find_nearest_plev_indices(plev=lat_xO  , target_levels=[-41.])
print(lat_xO[lat_sel])

In [ ]:
t,z,y,x = 2,lev_sel[4],lat_sel[0],-999
x0,x1=0,140
print(zlev[z] )

In [ ]:
MapProj =ccrs.PlateCarree() #ccrs.Orthographic(0, -90.)
DataProj = ccrs.PlateCarree()
# Get the name of the projection
proj_name = MapProj.__class__.__name__
levy=np.linspace(-10,10,num=41) #[-10,-5,-1,-.5,-.1,-.05,-.01,0,.01,.05,.1,.5,1,5,10]
volev = 0.0002 * np.linspace(-1,1,num=5)

#fig, ax = plt.subplots(figsize=(10,10) , projection=MapProj )
fig = plt.figure(figsize=( 20, 10 ))
Axes = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax = fig.add_axes( Axes ) #, projection=MapProj)
c2=ax.contourf( lon_xO, lat_xO, dog_xO[t,z,:,:] ,cmap='bwr',levels=0.1*levy,alpha=1 ,extend='both' )#, transform=DataProj )
c3=ax.contour( lon_xO, lat_xO, topo_xO[:,:] ,cmap='terrain',levels=[1,100,300,500,1000],alpha=.2 )#, transform=DataProj )
c4=ax.contour( lon_xO, lat_xO, zeta_xO[t,lev_sel[2],:,:] ,colors='black',levels=volev,alpha=.2 )#, transform=DataProj )

oo=r"$\omega'/g \rho$"
annot= ax.text( 0.05,.05,A.X.time[t].dt.strftime('%Y-%m-%d %HZ').values , transform=ax.transAxes, fontsize=20, bbox=dict(facecolor='white') )
annot2= ax.text( 0.4,.99, f'{oo} at Z={zlev[z]:.0f} km' , transform=ax.transAxes, fontsize=20, bbox=dict(facecolor='white') )
degree_symbol = '\u00B0'
ax.set_xlabel( f'Longitude ({degree_symbol}E)', fontsize=16)
ax.set_ylabel( f'Latitude ({degree_symbol})' ,fontsize=16 )

loo=ax.plot( [x0,x1], [0,0]+lat_xO[y])
cbar=plt.colorbar(c2,ax=ax,shrink=0.7,label=r"m s$^{-1}$")

In [ ]:
degree_symbol = '\u00B0'

from matplotlib.colors import BoundaryNorm

#co=plt.contourf( lon_xO, zlev, uu_xO[0,:,8*42,:] ,cmap='bwr')
levy=[-10,-5,-3,-1,-.5,-.1,-.05,-.01,0,.01,.05,.1,.5,1,3,5,10]
levy2=[-10,-1,-.1,-.01,0,.01,.1,1,10]
volev = 0.0002 * np.linspace(-1,1,num=51)
uulev=np.linspace(-20,20,num=11)

# Create a BoundaryNorm for uniform colors
norm = BoundaryNorm(levy, ncolors=256, clip=True)

# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(1, 3, figsize=(30, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()


n=0

c2=ax[n].contourf( Lon_xO[t,:,y,:] , z3o_xO[t,:,y,:] , dog_xO[t,:,y,:] ,cmap='bwr',levels=levy,norm=norm, alpha=1)
#c3=plt.contour(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , zeta_xO[t,:,y,:] ,colors='black',levels=21, alpha=0.2)
c3=ax[n].contour(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , dse_xO[t,:,y,:] ,colors='black',levels=cpair*270.+grav*np.linspace(0,15000,num=31) , alpha=0.2)
ax[n].plot( lon_xO, topo_xO[y,:]  ) 
ax[n].set_xlim((x0,x1))
ax[n].set_ylim(0,25_000)
ax[n].set_ylabel( 'Z(m)', fontsize=16)
ax[n].set_xlabel( f'Longitude ({degree_symbol}E)', fontsize=16)
plt.colorbar(c2,label=r'm s$^{-1}$')

n=1

c3=ax[n].contourf(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , zeta_xO[t,:,y,:] ,cmap='plasma',levels=volev , alpha=0.5)
#c2=ax[n].contour( Lon_xO[t,:,y,:] , z3o_xO[t,:,y,:] , dog_xO[t,:,y,:] ,colors='green',levels=levy2,norm=norm, alpha=.15)
c4=ax[n].contour(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , dse_xO[t,:,y,:] ,colors='black',levels=cpair*270.+grav*np.linspace(0,15000,num=31) , alpha=0.2)
ax[n].plot( lon_xO, topo_xO[y,:]  ) 
ax[n].set_xlim((x0,x1))
ax[n].set_ylim(0,25_000)
ax[n].set_ylabel( 'Z(m)', fontsize=16)
ax[n].set_xlabel( f'Longitude ({degree_symbol}E)', fontsize=16)
plt.colorbar(c3,label=r'm s$^{-1}$')

n=2

c3=ax[n].contourf(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , np.average(uu_xO[t,:,y-2*24:y+2*24,:],axis=1) ,cmap='bwr',levels=uulev , alpha=0.5)
#c2=ax[n].contour( Lon_xO[t,:,y,:] , z3o_xO[t,:,y,:] , dog_xO[t,:,y,:] ,colors='green',levels=levy2,norm=norm, alpha=.15)
c4=ax[n].contour(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , dse_xO[t,:,y,:] ,colors='black',levels=cpair*270.+grav*np.linspace(0,15000,num=31) , alpha=0.2)
ax[n].plot( lon_xO, topo_xO[y,:]  ) 
ax[n].set_xlim((x0,x1))
ax[n].set_ylim(0,25_000)
ax[n].set_ylabel( 'Z(m)', fontsize=16)
ax[n].set_xlabel( f'Longitude ({degree_symbol}E)', fontsize=16)
plt.colorbar(c3,label=r'm s$^{-1}$')



In [ ]:
from matplotlib.colors import BoundaryNorm
fig = plt.figure(figsize=( 10, 5 ))
Axes = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax = fig.add_axes( Axes )

#co=plt.contourf( lon_xO, zlev, uu_xO[0,:,8*42,:] ,cmap='bwr')
levy=[-10,-5,-3,-1,-.5,-.1,-.05,-.01,0,.01,.05,.1,.5,1,3,5,10]

# Create a BoundaryNorm for uniform colors
norm = BoundaryNorm(levy, ncolors=256, clip=True)

c2=ax.contourf( Lon_xO[t,:,y,:] , z3o_xO[t,:,y,:] , dog_xO[t,:,y,:] ,cmap='bwr',levels=levy,norm=norm, alpha=1)
#c3=plt.contour(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , zeta_xO[t,:,y,:] ,colors='black',levels=21, alpha=0.2)
c3=ax.contour(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , dse_xO[t,:,y,:] ,colors='black',levels=cpair*270.+grav*np.linspace(0,15000,num=31) , alpha=0.2)
ax.plot( lon_xO, topo_xO[y,:]  ) 
ax.set_xlim((x0,x1))
ax.set_ylim(0,25_000)
ax.set_ylabel( 'Z(m)', fontsize=16)
degree_symbol = '\u00B0'
ax.set_xlabel( f'Longitude ({degree_symbol}E)', fontsize=16)
plt.colorbar(c2,label=r'm s$^{-1}$')

In [ ]:
from matplotlib.colors import BoundaryNorm
fig = plt.figure(figsize=( 10, 5 ))
Axes = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax = fig.add_axes( Axes )

#co=plt.contourf( lon_xO, zlev, uu_xO[0,:,8*42,:] ,cmap='bwr')
levy=[-10,-5,-3,-1,-.5,-.1,-.05,-.01,0,.01,.05,.1,.5,1,3,5,10]
levy2=[-10,-1,-.1,-.01,0,.01,.1,1,10]
volev = 0.0002 * np.linspace(-1,1,num=51)

# Create a BoundaryNorm for uniform colors
norm = BoundaryNorm(levy, ncolors=256, clip=True)

#c2=ax.contourf( Lon_xO[t,:,y,:] , z3o_xO[t,:,y,:] , dog_xO[t,:,y,:] ,cmap='bwr',levels=levy,norm=norm, alpha=.5)
#c3=plt.contour(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , zeta_xO[t,:,y,:] ,colors='black',levels=21, alpha=0.2)
c3=ax.contourf(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , zeta_xO[t,:,y,:] ,cmap='plasma',levels=volev , alpha=0.5)
#c2=ax.contour( Lon_xO[t,:,y,:] , z3o_xO[t,:,y,:] , dog_xO[t,:,y,:] ,colors='green',levels=levy2,norm=norm, alpha=.15)
c4=ax.contour(  Lon_xO[t,:,y,:] ,  z3o_xO[t,:,y,:] , dse_xO[t,:,y,:] ,colors='black',levels=cpair*270.+grav*np.linspace(0,15000,num=31) , alpha=0.2)
ax.plot( lon_xO, topo_xO[y,:]  ) 
ax.set_xlim((x0,x1))
ax.set_ylim(0,25_000)
ax.set_ylabel( 'Z(m)', fontsize=16)
degree_symbol = '\u00B0'
ax.set_xlabel( f'Longitude ({degree_symbol}E)', fontsize=16)
plt.colorbar(c3,label=r'm s$^{-1}$')

In [ ]:
print(uu_xO.shape)